### Simulating a FHN network.

In [15]:
include("src/network_matrices_creation.jl")
include("src/network_simulation.jl")

# Define all the parameters
N = 90
eps = 0.05
a = 0.5
b = bmatrix(pi/2-0.1, eps)
σ = 0.0506
G = wattsstrogatzmatrix(N, 3, 1)#0.232)

# Set the initial conditions
x_0 = zeros(2*N)
x_0[1:2] .+= 0.1

# Define the ODE problem
prob = ODEProblem((dx, x, params, t) -> coupled_fhn_eom!(dx, x, params[1], params[2], params[3], G, b), x_0, (0.0, 300.0), [a, eps, σ])

# Solve the ODE problem
# alg = Tsit5()
sol = solve(prob; dtmax=0.1);

In [44]:
include("src/network_sol_analysis.jl")
using GLMakie
using LaTeXStrings

f = Figure(resolution = (1500, 1000))
ax = Axis(f[1, 1])
ax.title = "Kuramoto Order Parameter"
ax.xlabel = "Time"
ax.ylabel = "Kuramoto Order Parameter"
t_val, kuramoto_val = kuramoto_time_series(sol, N)
lines!(ax, t_val, kuramoto_val)
f


### Studying Cluster Synchronization in the integration of the FHN model.

In [17]:
using GLMakie

include("src/cluster_synch.jl")
include("src/network_sol_analysis.jl")
eigenvectors, eigenvalues, clusters, s_matrices = s_matrix_method(G)

f = Figure(resolution = (1500, 1000))
ax = Axis(f[1, 1])
ax.title = "Synch Error by Cluster"
ax.xlabel = "Time"
ax.ylabel = "Synch Error"

i = 0
for cluster in clusters
    if isempty(cluster)
        continue
    else
        n_cluster = i
        t_values, synch_error = local_synch_error(sol, cluster)
        lines!(ax, t_values, synch_error; label="Cluster $n_cluster") # Label = "Cluster $cluster"
        i += 1
    end

end
axislegend()
f

